# __利用Pandas的套件pandasql的sqldf來寫sql__

In [1]:
import pandas as pd
from pandasql import sqldf

In [2]:
file_path_staff = "/workspaces/tir102_data-pipeline/resource/staff.csv"
file_path_staff_detail = "/workspaces/tir102_data-pipeline/resource/staff_detail.csv"

df_staff = pd.read_csv(file_path_staff)
df_staff_detail = pd.read_csv(file_path_staff_detail)

In [3]:
sql = """
    select * from df_staff
"""

sqldf(sql)

,ID,Name,DeptId,Age,Gender,Salary
0,1,Mike,2,45,M,60000
1,2,Judy,2,30,F,48000
2,3,Allen,1,22,M,50000
3,4,Tom,2,47,M,47000
4,5,Jack,3,36,M,52000
5,6,Abby,2,24,F,45000
6,7,Trump,1,80,M,80000
7,8,Marry,3,29,F,87000


In [4]:
sql = """
    select * 
    from df_staff_detail sd
    left join df_staff s
    on sd.ID = s.ID
"""

sqldf(sql)

,ID,Level,UpdateDate,ID,Name,DeptId,Age,Gender,Salary
0,1,A,2019-08-09,1,Mike,2,45,M,60000
1,1,S,2020-12-08,1,Mike,2,45,M,60000
2,2,A,2021-05-30,2,Judy,2,30,F,48000
3,3,A,2018-03-26,3,Allen,1,22,M,50000
4,3,S,2020-12-08,3,Allen,1,22,M,50000
5,4,A,2022-05-02,4,Tom,2,47,M,47000
6,5,A,2019-11-05,5,Jack,3,36,M,52000
7,5,S,2022-04-25,5,Jack,3,36,M,52000
8,6,A,2022-03-03,6,Abby,2,24,F,45000
9,7,A,2017-03-29,7,Trump,1,80,M,80000


In [9]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'import pandas as pd\nfrom pandasql import sqldf',
  'file_path_staff = "/workspaces/tir102_data-pipeline/resource/staff.csv"\nfile_path_staff_detail = "/workspaces/tir102_data-pipeline/resource/staff_detail.csv"\n\ndf_staff = pd.read_csv(file_path_staff)\ndf_staff_detail = pd.read_csv(file_path_staff_detail)',
  'sql = """\n    select * from df_staff\n"""\n\nsqldf(sql)',
  'sql = """\n    select * \n    from df_staff_detail sd\n    left join df_staff s\n    on sd.ID = s.ID\n"""\n\nsqldf(sql)',
  'globals()',
  'globals("staff_detail")',
  'globals("staff_detail.csv")',
  'globals(file_path_staff)',
  'globals()'],
 '_oh': {3:    ID   Name  DeptId  Age Gender  Salary
  0   1   Mike       2   45      M   60000
  

In [10]:
columns = ["article", "author", "price"]
data = [
    ["0001", "B", 3.99],
    ["0002", "A", 10.99],
    ["0003", "C", 1.69],
    ["0004", "B", 19.95],
    ["0005", "A", 6.96],
]
prticle_price_df = pd.DataFrame(data=data, columns=columns)
prticle_price_df

,article,author,price
0,0001,B,3.99
1,0002,A,10.99
2,0003,C,1.69
3,0004,B,19.95
4,0005,A,6.96


In [11]:
temp_sql = """
select
        *,                 #partition: 先退author做groupby 再去對price做降續
        ROW_NUMBER() OVER (PARTITION BY author ORDER BY price DESC) as ROW_ID
    from prticle_price_df
"""

sqldf(temp_sql)

,article,author,price,ROW_ID
0,0002,A,10.99,1
1,0005,A,6.96,2
2,0004,B,19.95,1
3,0001,B,3.99,2
4,0003,C,1.69,1


In [15]:
# 先把下面的table把它做成一個暫時的table(子查詢的延伸應用;若子查詢的邏輯太大 會先把某些東西變成一個暫時的table)
sql = """
with tmp_prticle_price_df as (
    select
        *,
        ROW_NUMBER() OVER (PARTITION BY author ORDER BY price DESC) as ROW_ID
    from prticle_price_df
)
select
    article,
    author,
    price
from tmp_prticle_price_df
where ROW_ID = 1
"""
sqldf(
    sql,
)

,article,author,price
0,0002,A,10.99
1,0004,B,19.95
2,0003,C,1.69
